## Advanced NN

### Import Module

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
from tqdm import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import OrderedDict
from IPython.display import clear_output

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.autograd import Variable
import tensorflow as tf

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

### Data

#### label 
- 0 :T-shirt/top
- 1 : Trouser
- 2 : Pullover 
- 3 : Dress
- 4 : Coat 
- 5 : Sandal 
- 6 : Shirt 
- 7 : Sneaker 
- 8 : Bag 
- 9 : Ankle boot

In [3]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [4]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,67,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,2,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,93,58,78,45,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,71,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,63,0,0,0,0,0,0,0


#### Split Data

In [5]:
X_train = train_df.iloc[:,1:].values / 255
y_train = train_df['label'].values

In [6]:
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

#### transforms into numpy

In [7]:
X_train_torch = torch.from_numpy(X_train).type(torch.FloatTensor)
X_dev_torch = torch.from_numpy(X_dev).type(torch.FloatTensor)
y_train_torch = torch.from_numpy(y_train).type(torch.LongTensor)
y_dev_torch = torch.from_numpy(y_dev).type(torch.LongTensor)

#### dimension of data

In [8]:
X_train_torch = X_train_torch.view(-1,1,28,28).float()
X_dev_torch = X_dev_torch.view(-1,1,28,28).float()

#### make tensor datasets

In [9]:
train_set = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)
valid_set = torch.utils.data.TensorDataset(X_dev_torch, y_dev_torch)

#### make data loaders for mini-batch

In [10]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(train_set, shuffle = True, batch_size = 128)
valid_loader = torch.utils.data.DataLoader(valid_set, shuffle = True, batch_size = 128)

#### check GPU

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
device

device(type='cpu')

### Train

In [12]:
list_process = []

In [13]:
def train(model, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for data, label in train_loader:
        data, label = data.to(device), label.to(device)  
        optimizer.zero_grad()  
        output = model(data) 
        loss = criterion(output, label)  
        loss.backward() 
        optimizer.step()  
        train_loss += loss.item() 
        #get argmax values in outputs
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(label.view_as(pred)).sum().item()
    print('epoch for train: {}, accuracy: ({:.2f}%)'.format(epoch,correct*100 / len(train_loader.dataset)))
    list_process.append(correct*100 / len(train_loader.dataset))

### Evaluate

In [14]:
def valid(model, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, label in valid_loader:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += criterion(output, label).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()
    print('epoch for test: {}, accuracy: ({:.2f}%)'.format(epoch,correct*100 / len(valid_loader.dataset)))

### Model : CNN

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [16]:
model = CNN()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss() 

In [17]:
print(model)
for epoch in tqdm(range(10)):
    train(model,epoch)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


 10%|████▍                                       | 1/10 [01:03<09:31, 63.52s/it]

epoch for train: 0, accuracy: (80.18%)


 20%|████████▊                                   | 2/10 [02:08<08:33, 64.21s/it]

epoch for train: 1, accuracy: (86.66%)


...

epoch for train: 7, accuracy: (92.51%)


 90%|███████████████████████████████████████▌    | 9/10 [09:33<01:03, 63.78s/it]

epoch for train: 8, accuracy: (92.85%)


100%|███████████████████████████████████████████| 10/10 [10:36<00:00, 63.64s/it]

epoch for train: 9, accuracy: (93.09%)


In [18]:
x_test = test_df.values / 255
x_test_torch = torch.from_numpy(x_test).type(torch.FloatTensor)
d_labels = np.zeros(x_test.shape)
d_labels = torch.from_numpy(d_labels)

x_test_torch = x_test_torch.view(-1, 1, 28, 28)
testset = torch.utils.data.TensorDataset(x_test_torch, d_labels)
testloader = torch.utils.data.DataLoader(testset, batch_size = 1, shuffle = False)

In [19]:
submit_list = [['id', 'label']]
with torch.no_grad():
    model.eval()
    image_id = 0
    for images,label in testloader:
        images,label = images.to(device), label.to(device)
        outputs = model(images)
        probs = torch.exp(outputs)
        top_p, top_class = probs.topk(1, dim = 1)
        for preds in top_class:
            submit_list.append([image_id,preds.item()])
            image_id += 1

In [20]:
df = pd.DataFrame(submit_list)
df.columns = df.iloc[0]
df = df.drop(0, axis = 0)
df.to_csv('submit.csv', index = False)
print("submit.csv saved")

submit.csv saved
